In [1]:
from openff.toolkit import ForceField, Molecule, unit
from openff.interchange.components._packmol import UNIT_CUBE, pack_box
from copy import deepcopy
import nglview
import mdtraj
import numpy as np
from openff.interchange import Interchange
import openmm
import openmm.app
import openmm.unit
import time

/home/mmh/micromamba/envs/opmdf-ubuntu-latest-bespokefit-false/lib/python3.12/site-packages/nglview/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Define molecules, mapped smiles are useful so the index stays the same

In [2]:
ibuprofen_smiles = "[C:1]([C:2]([C:3]([H:20])([H:21])[H:22])([C:4]([c:5]1[c:6]([H:25])[c:7]([H:26])[c:8]([C@@:11]([C:12]([H:30])([H:31])[H:32])([C:13](=[O:14])[O:15][H:33])[H:29])[c:9]([H:27])[c:10]1[H:28])([H:23])[H:24])[H:19])([H:16])([H:17])[H:18]"
water_smiles = "[H:2][O:1][H:3]"

Create the water molecule and add some metadata to make the viz better

In [3]:
water = Molecule.from_mapped_smiles(water_smiles)
water.generate_conformers(n_conformers=1)
for atom in water.atoms:
    atom.metadata["residue_name"] = "HOH"

Create the ibuprofen molecule

In [4]:
ibuprofen = Molecule.from_mapped_smiles(ibuprofen_smiles)
ibuprofen.generate_conformers(n_conformers=1)

Create a box of 500 ibuprofen molecules at a density of 1.2 grams per cubic centimeter

In [5]:
n_ibuprofen = 500
target_density = 1.2 * unit.gram / unit.centimetre**3

topology = pack_box(
    molecules=[ibuprofen],
    number_of_copies=[n_ibuprofen],
    target_density=target_density,
    box_shape=UNIT_CUBE,
    working_directory=".",
)

Visualize the box

In [6]:
topology.to_file("system.pdb")
nglview.show_structure_file("system.pdb")

NGLWidget()

Now create a box of water at the same target density and size. With a little math, we can calculate the number of water molecules.

In [18]:
target_density = 1.2 * unit.gram / unit.centimetre**3
# Since we are using a cube, we can calc the vol from the length of the topo from the ibuprofen box
box_vol = topology.box_vectors[0][0] ** 3
mass = sum([atom.mass for atom in water.atoms])
n_water = int(box_vol * target_density / mass)
water_topology = pack_box(
    molecules=[water],
    number_of_copies=[n_water],
    box_vectors=topology.box_vectors,
    working_directory=".",
)

Now we can look at the water box,we need to configure nglview a bit so we can see the water (water is hidden by default).

In [19]:
water_topology.to_file("system.pdb")
view = nglview.show_mdtraj(mdtraj.load("system.pdb"))
view.add_representation("licorice", selection="water")
view

NGLWidget()

Now we will combine the topology by shifting the water positions by the length of the ibuprofen box (we make a copy so if we re-run this cell, we can use the original positions). We then update the box vectors so we go from a cube to a rectangle. We then visualize the results.

In [21]:
water_topology_copy = deepcopy(water_topology)
water_topology_copy.set_positions(
    water_topology.get_positions() + topology.box_vectors[0]
)
combined = water_topology_copy + topology
combined.box_vectors = topology.box_vectors * np.array(
    [[2, 0, 0], [0, 1, 0], [0, 0, 1]]
)
combined.to_file("system.pdb")
view = nglview.show_mdtraj(mdtraj.load("system.pdb"))
view.add_representation("licorice", selection="water")
view

NGLWidget()

Now we create our interchange object and parameterize our system.

In [22]:
sage = ForceField("openff-2.2.1.offxml")
interchange = sage.create_interchange(combined)

Visualize things one more time to make sure our box looks correct

In [24]:
interchange.visualize()

NGLWidget()

Create some helper functions for our simulation

In [25]:
def create_simulation(
    interchange: Interchange,
    pdb_stride: int = 500,
    trajectory_name: str = "trajectory.pdb",
) -> openmm.app.Simulation:
    integrator = openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    barostat = openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar,
        293.15 * openmm.unit.kelvin,
        25,
    )

    simulation = interchange.to_openmm_simulation(
        combine_nonbonded_forces=True,
        integrator=integrator,
        additional_forces=[barostat],
    )

    # https://github.com/openmm/openmm/issues/3736#issuecomment-1217250635
    simulation.minimizeEnergy()

    simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
    simulation.context.computeVirtualSites()

    pdb_reporter = openmm.app.PDBReporter(trajectory_name, pdb_stride)
    state_data_reporter = openmm.app.StateDataReporter(
        "data.csv",
        10,
        step=True,
        potentialEnergy=True,
        temperature=True,
        density=True,
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation


def run_simulation(simulation: openmm.app.Simulation, n_steps: int = 5000):
    print("Starting simulation")
    start_time = time.process_time()

    print("Step, volume (nm^3)")

    for step in range(n_steps):
        simulation.step(1)
        if step % 500 == 0:
            box_vectors = simulation.context.getState().getPeriodicBoxVectors()
            print(step, np.linalg.det(box_vectors._value).round(3))

    end_time = time.process_time()
    print(f"Elapsed time: {(end_time - start_time):.2f} seconds")

Create our simulation object (which does do some minimization so it takes some time)

In [27]:
simulation = create_simulation(interchange)

Now we run our simulation

In [29]:
run_simulation(simulation)

Starting simulation
Step, volume (nm^3)
0 379.933
500 392.587
1000 406.888
1500 409.093
2000 416.37
2500 419.488
3000 420.389
3500 420.649
4000 421.549
4500 421.901
Elapsed time: 9.50 seconds


Now we visualize the results

In [30]:
view = nglview.show_mdtraj(mdtraj.load("trajectory.pdb"))
view.add_representation("licorice", selection="water")
view

NGLWidget(max_frame=9)